In [1]:
%matplotlib widget
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker, cm
from matplotlib.ticker import LogFormatter 

from scipy.optimize import curve_fit
import copy
import os
import h5py
import json

import PyPDF2                       # Per unire i PDF alla fine
import glob                         # Per manipolazione dei path

from collections.abc import Iterable


In [5]:
#config_file = r"./config.json"

#with open(config_file, "r") as f:
#        dizi = json.load(f)
#dizi
Swan = True

DEFS

In [6]:
def myGauss(x, a, mu, sigma):
    return a * np.exp(- (x-mu)**2 / (2*sigma**2))

def myGauss_line(x, a, mu, sigma,m,q):
    return a * np.exp(- (x-mu)**2 / (2*sigma**2)) + (m*x + q)

# def project_to_z_pos(z1,z2,z4):
def projectDistZ(z):
    
    mx = (x2-x1)/dizi['d_12']
    xProj = x1 + mx * z
    
    my = (y2-y1)/dizi['d_12']
    yProj = y1 + my * z
    
    return (xProj, yProj)


def Average(lst):
    return sum(lst) / len(lst)

In [12]:
def file_corrector(runs):
    if Swan:
        data_dir = "/eos/project/i/insulab-como/testBeam/TB_T9_2023_08_OREO/HDF5"
    else:
        data_dir = "."

    pos = []
    infos = []
    phs = []
    tmis = []
    evis =[]

    if not isinstance(runs, Iterable):
        runs = [runs]

    for run in runs:
        data_path = f'{data_dir}/run{run}.h5'

        with h5py.File(data_path, 'r', libver='latest', swmr=True) as hf:
            print(hf.keys())
            hf["xpos"].shape
            keys = list(hf.keys())
            #for k in hf.keys():
            #    comand = f'{k} = np.array(hf["{k}"])'
                # print(comand)
            #  exec(comand)
            pos.append(np.array(hf['xpos']))
            infos.append(np.array(hf['xinfo']))
            phs.append(np.array(hf['digiPH']))
            tmis.append(np.array(hf['digiTime']))
            evis.append(np.array(hf['ievent']))

    #print(np.shape(pos))
    # print(np.shape(infos))
            
    xpos = np.concatenate(pos,axis=0)
    xinfo = np.concatenate(infos,axis=0)
    ph = np.concatenate(phs,axis=0)
    tm = np.concatenate(tmis,axis=0)
    evi = np.concatenate(evis,axis=0)

    #print(np.shape(xpos))
    # print(np.shape(xinfo))

    ##purge errors
    logic = (xpos > -1) & (xpos < 15)
    logic2 = logic.all(axis = 1)
    xpos = xpos[logic2]   

    xinfo = xinfo[logic2]
    ph = ph[logic2]
    tm = tm[logic2]
    evi = evi[logic2]
        #NO CORRECTION IN EQUALIZER!
    George = (ph[:,2]+ph[:,3])#*dizi['George_correction']
    John = (ph[:,4]+ph[:,5])#*dizi['John_correction']
    Paul = (ph[:,6]+ph[:,7])#*dizi['Paul_correction']
    Beatles = np.array([George,John,Paul]).T

    xpos[:,2] -= dizi['offset_x2']
    xpos[:,3] -= dizi['offset_y2']

    x1 = xpos[:,0]
    y1 = xpos[:,1]
    x2 = xpos[:,2] 
    y2 = xpos[:,3] 
    x_cry, y_cry = projectDistZ(dizi['d_1c'])
    return xpos,xinfo,ph,tm,evi,Beatles,x1,y1,x2,y2,x_cry,y_cry 

In [11]:
n_runs  = [680253,680254,680258,680259,680260,680261,680256,680257]
Beam_en = [1,2,3,4,5,6,12,12]
xpos,xinfo,ph,tm,evi,Beatles,x1,y1,x2,y2,x_cry,y_cry = file_corrector(n_runs)

print(xpos.shape)




<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>
<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>
<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>
<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>
<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>
<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>
<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>
<KeysViewHDF5 ['digiBase', 'digiPH', 'digiTime', 'ievent', 'info_plus', 'nclu', 'nstrip', 'xinfo', 'xpos']>


NameError: name 'dizi' is not defined